# Gradio Chatbot UI for Multi-Agent System
Interact with your AI agents using a conversational interface powered by Gradio and the logic in `mas.py`.

In [1]:
import os
import asyncio
import gradio as gr
from src.mas import Orchestrator

# Initialize the orchestrator
orchestrator = Orchestrator()

# Global chat history
chat_history = []

## Define the Chatbot Logic
This function will be called on each user message. It sends the message to the orchestrator and returns the updated chat history.

In [2]:
def chat_with_agents(user_message, history):
    global chat_history
    if not user_message.strip():
        return history, ""
    # Append user message to history for display
    history = history or []
    history.append(("user", user_message))
    # Run orchestrator and get agent responses
    try:
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        messages = loop.run_until_complete(orchestrator.run(user_message, chat_history))
        # Extract agent responses and update history
        for msg in messages[len(chat_history):]:
            role = getattr(msg, 'role', None) or getattr(msg, 'author_role', None) or 'assistant'
            content = getattr(msg, 'content', str(msg))
            if role == 'user':
                continue
            history.append((role, content))
        chat_history = messages
    except Exception as e:
        history.append(("system", f"Error: {e}"))
    return history, ""

## Build the Gradio UI
This UI includes a chatbot window and a textbox for user input.

In [3]:
with gr.Blocks(title="Multi-Agent Chatbot") as demo:
    gr.Markdown("""# Multi-Agent Chatbot\nChat with your AI agents below.""")
    chatbot = gr.Chatbot(label="AI Chatbot", height=400)
    textbox = gr.Textbox(placeholder="Type your message and press Enter", show_label=False, lines=1)

    def clear_history():
        global chat_history
        chat_history = []
        return []

    textbox.submit(chat_with_agents, [textbox, chatbot], [chatbot, textbox])
    gr.Button("Clear").click(clear_history, outputs=chatbot)

if __name__ == "__main__":
    demo.launch()

/tmp/ipykernel_3630532/3226252814.py:3: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="AI Chatbot", height=400)


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


# AuthorRole.USER: 'Help me to interpret the AI model's results /use_reasoning'
# AuthorRole.ASSISTANT - ReasoningAgent: '### Interpretation of AI Model's Results Using SHAP Values

Below is a breakdown of the SHAP values for different instances, highlighting key features, their contributions, feature interactions, and actionable insights.

---

## Instance 1: Predicted Sales = 7.9968047
- **Key Influential Features** (in descending order of absolute SHAP values):  
  - **`sales_rolling_mean_20`**: SHAP Value = 4.216077, Contribution = Positive.  
    Explanation: A high 20-day rolling average suggests sustained demand, leading to a significant boost in predicted sales. This feature is highly correlated with recent purchasing trends.  
  - **`specific_day_sales_average`**: SHAP Value = 0.40363818, Contribution = Positive.  
    Explanation: Strong average sales on the specific day positively impact the prediction, reflecting consumer behavior on that particular day.  
  - **`sales_expa